# MobileNet Model without Data Augmentation 

In [1]:
# Pre trained MobileNet weight 
# RGB Image 
# The original size of the input image for the pretrained MobileNet model is 224x224 (224x224x3) 
# The size of our input image is resized to 128x128 (128x128x3)

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Activation
from keras.layers import Input
from keras.utils import multi_gpu_model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.layers import Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image 
from keras.models import Model

Using TensorFlow backend.


###### Loading data 
Intially the datas are structured to input to the convolutional layer by converting the images in to numpy array, normalized (changing the range of pixel intensity values) and reshaped accordingly. Finally, stored in the path shown below.

In [3]:
x_train = np.load('/cai_lab/thomas/DWD/RGB_128_3Channel_Dataset/train/x_data_all.npy')
y_train = np.load('/cai_lab/thomas/DWD/RGB_128_3Channel_Dataset/train/y_data_all.npy')

x_valid = np.load('/cai_lab/thomas/DWD/RGB_128_3Channel_Dataset/valid/x_data_all.npy')
y_valid = np.load('/cai_lab/thomas/DWD/RGB_128_3Channel_Dataset/valid/y_data_all.npy')

print('x_train.shape:',x_train.shape,'y_train.shape:',y_train.shape)
print('x_valid.shape:',x_valid.shape,'y_valid.shape:',y_valid.shape)

x_train.shape: (17194, 128, 128, 3) y_train.shape: (17194, 9)
x_valid.shape: (2558, 128, 128, 3) y_valid.shape: (2558, 9)


In [4]:
batch_size = 24
resize = 128    
n_class = 9


## MobileNet Model 

In [5]:
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

inputs = Input (shape = (resize, resize, 3))


MobileNet_model=MobileNet(weights='imagenet',include_top=False,  input_shape= (224,224,3)) 
MobileNet_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [6]:
# Changing the fixed input of mobilenet to the reduced image size input 

output_mobileNet = MobileNet_model(inputs)

y=GlobalAveragePooling2D()(output_mobileNet)


dense_layer = Dense(1024, activation = 'relu')(y)
dense_layer = Dropout(0.5)(dense_layer)
dense_layer = Dense(1024, activation = 'relu')(dense_layer)
dense_layer = BatchNormalization()(dense_layer)
dense_layer = Dropout(0.5)(dense_layer)
dense_layer = Dense(512, activation = 'relu')(dense_layer)

final_output = Dense(n_class, activation = 'softmax')(dense_layer)


final_model = Model ([inputs], final_output)

final_model.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
mobilenet_1.00_224 (Model)   multiple                  3228864   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096

### Model Saving and Compling 

In [7]:

save_name = 'MobileNet_RGB_128'
save_path = './checkpoints/' + save_name

if os.path.exists(save_path)==False:
    os.makedirs(save_path)

checkpoint = ModelCheckpoint(save_path +'/checkpoint-{epoch:04d}.h5', verbose=1, monitor='val_loss', save_best_only=False, mode='auto', period=1)
tensorboard = TensorBoard(log_dir='./graph/'+ save_name, histogram_freq=0, write_graph=True, write_images=True)
callback_list = [checkpoint, tensorboard]

final_model = multi_gpu_model(final_model, gpus=2)

# aug = ImageDataGenerator(
#         rotation_range=20,
#         zoom_range=0.15,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.15,
#         horizontal_flip=True,
#         fill_mode="nearest")


# Compiling
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)
sgd = optimizers.SGD(lr = 0.001)

final_model.compile(loss='categorical_crossentropy',optimizer = sgd,metrics=['accuracy'])


final_model.fit(x_train, y_train, 
                     batch_size=batch_size,
                    validation_data=(x_valid, y_valid),       
                    epochs=60,
                    verbose=1,
                    callbacks=callback_list)

final_model.save(save_path + '/epoch-finish.h5')
final_model.save_weights(save_path +'/epoch-finish_weights.h5')

W0522 15:11:50.304518 140175971358464 deprecation.py:323] From /home/cai/.local/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0522 15:11:56.131942 140175971358464 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0522 15:11:56.145108 140175971358464 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0522 15:11:56.146046 140175971358464 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras

Train on 17194 samples, validate on 2558 samples
Epoch 1/60
17194/17194 [==============================] - 51s 3ms/step - loss: 1.9229 - accuracy: 0.3567 - val_loss: 1.4768 - val_accuracy: 0.5039

Epoch 00001: saving model to ./checkpoints/Comp_MobileNet_RGB_128/checkpoint-0001.h5


W0522 15:12:54.892135 140175971358464 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:343: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



Epoch 2/60
17194/17194 [==============================] - 41s 2ms/step - loss: 0.8433 - accuracy: 0.7182 - val_loss: 1.1305 - val_accuracy: 0.6372

Epoch 00002: saving model to ./checkpoints/Comp_MobileNet_RGB_128/checkpoint-0002.h5
Epoch 3/60
17194/17194 [==============================] - 41s 2ms/step - loss: 0.4669 - accuracy: 0.8504 - val_loss: 1.0554 - val_accuracy: 0.6665

Epoch 00003: saving model to ./checkpoints/Comp_MobileNet_RGB_128/checkpoint-0003.h5
Epoch 4/60
17194/17194 [==============================] - 41s 2ms/step - loss: 0.3141 - accuracy: 0.8988 - val_loss: 1.1110 - val_accuracy: 0.6630

Epoch 00004: saving model to ./checkpoints/Comp_MobileNet_RGB_128/checkpoint-0004.h5
Epoch 5/60
17194/17194 [==============================] - 41s 2ms/step - loss: 0.2268 - accuracy: 0.9305 - val_loss: 1.0852 - val_accuracy: 0.6626

Epoch 00005: saving model to ./checkpoints/Comp_MobileNet_RGB_128/checkpoint-0005.h5
Epoch 6/60
17194/17194 [==============================] - 41s 2ms/st

17194/17194 [==============================] - 41s 2ms/step - loss: 0.0160 - accuracy: 0.9958 - val_loss: 1.8862 - val_accuracy: 0.6142

Epoch 00037: saving model to ./checkpoints/Comp_MobileNet_RGB_128/checkpoint-0037.h5
Epoch 38/60
17194/17194 [==============================] - 41s 2ms/step - loss: 0.0146 - accuracy: 0.9959 - val_loss: 1.9360 - val_accuracy: 0.6087

Epoch 00038: saving model to ./checkpoints/Comp_MobileNet_RGB_128/checkpoint-0038.h5
Epoch 39/60
17194/17194 [==============================] - 41s 2ms/step - loss: 0.0143 - accuracy: 0.9963 - val_loss: 1.9169 - val_accuracy: 0.6118

Epoch 00039: saving model to ./checkpoints/Comp_MobileNet_RGB_128/checkpoint-0039.h5
Epoch 40/60
17194/17194 [==============================] - 41s 2ms/step - loss: 0.0138 - accuracy: 0.9959 - val_loss: 1.9387 - val_accuracy: 0.6063

Epoch 00040: saving model to ./checkpoints/Comp_MobileNet_RGB_128/checkpoint-0040.h5
Epoch 41/60
17194/17194 [==============================] - 41s 2ms/step - lo